In [0]:
import numpy
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense

In [0]:
import shutil
import os

# Каталог с набором данных
data_dir = 'train'
# Каталог с данными для обучения
train_dir = 'DBD/images/train'
# Каталог с данными для проверки
val_dir = 'DBD/images/validation'
# Каталог с данными для тестирования
test_dir = 'DBD/images/test'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Количество элементов данных в одном классе
nb_images = 12500

In [0]:
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 10
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750

Слои с 1 по 6 используются для выделения важных признаков в изображении, а слои с 7 по 10 - для классификации.

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) # Преобразование в одномерный вектор
model.add(Dense(64)) 
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1)) # На выходе 1-н нейрон
model.add(Activation('sigmoid')) # Данная функция характерна при бинарной классификации

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:
train_generator = datagen.flow_from_directory(
    train_dir, target_size=(img_width, img_height),
    batch_size=batch_size,class_mode='binary')

Found 17500 images belonging to 2 classes.


In [0]:
val_generator = datagen.flow_from_directory(
    val_dir, target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3750 images belonging to 2 classes.


In [0]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3750 images belonging to 2 classes.


In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=20, validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/20
1750/1750 [==============================] - 81s 46ms/step - loss: 0.6472 - acc: 0.6275 - val_loss: 0.5866 - val_acc: 0.6933
Epoch 2/20
1750/1750 [==============================] - 77s 44ms/step - loss: 0.5519 - acc: 0.7227 - val_loss: 0.4982 - val_acc: 0.7552
Epoch 3/20
1750/1750 [==============================] - 77s 44ms/step - loss: 0.4712 - acc: 0.7801 - val_loss: 0.4268 - val_acc: 0.7971
Epoch 4/20
1750/1750 [==============================] - 77s 44ms/step - loss: 0.4123 - acc: 0.8122 - val_loss: 0.4100 - val_acc: 0.8067
Epoch 5/20
1750/1750 [==============================] - 77s 44ms/step - loss: 0.3622 - acc: 0.8423 - val_loss: 0.3915 - val_acc: 0.8208
Epoch 6/20
1750/1750 [==============================] - 77s 44ms/step - loss: 0.3112 - acc: 0.8682 - val_loss: 0.4026 - val_acc: 0.8272
Epoch 7/20
1750/1750 [==============================] - 77s 44ms/step - loss: 0.2655 - acc: 0.8854 - val_loss: 0.4924 - val_acc: 0.8192
Epoch 8/20
1750/1750 [==========================

In [0]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [0]:
print("Точность на тестовых данных: {} %".format(scores[1]*100))

Точность на тестовых данных: 83.52000117301941 %


Сохраним нашу модель

In [0]:
model.save('DBD/binary.h5')

In [0]:
#  Загрузка файла
# from google.colab import files
# files.upload()

Предсказание модели

In [0]:
img = image.load_img('DBD/cat.jpg', target_size=(150, 150))
x = image.img_to_array(img)
x = numpy.expand_dims(x, axis=0)
x = preprocess_input(x)
predict = model.predict(x)

In [0]:
print("Результаты распознавания: ", predict) # Кошка

Результаты распознавания:  [[0.]]


In [0]:
img = image.load_img('DBD/dog.jpg', target_size=(150, 150))
x = image.img_to_array(img)
x = numpy.expand_dims(x, axis=0)
x = preprocess_input(x)
predict = model.predict(x)
print("Результаты распознавания: ", predict)

Результаты распознавания:  [[1.]]


In [0]:
int(predict[0][0]) # Собака

1